<a href="https://colab.research.google.com/github/nntadotzip/vi-augmentation/blob/main/BERT_SentencePairClassification_OriginalDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets==1.0.1
!pip install transformers==3.1.0

In [2]:
import torch
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import copy
import torch.optim as optim
import random
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
from datasets import load_dataset, load_metric

os.environ["TOKENIZERS_PARALLELISM"] = "false"

PyTorch version 1.10.0+cu111 available.
TensorFlow version 2.7.0 available.


In [3]:
# Check that we are using 100% of GPU memory footprint support libraries/code
# from https://github.com/patrickvonplaten/notebooks/blob/master/PyTorch_Reformer.ipynb
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip -q install gputil
!pip -q install psutil
!pip -q install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 11.9 GB  | Proc size: 1.6 GB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [4]:
df = pd.read_csv('pair-backtranslations_init277_to518_vi-en-fr_withCosineUsingSBert.csv', encoding='utf-8')
# df['idx'] = df.index
df

,question_1,question_2,similarity,sem_sim,q1_en_to_vi,q2_fr_to_vi,cosine_usingSBert
0,"""WTO là tên viết tắt của tổ chức nào sau đây?\...","""WTO là tên viết tắt của tổ chức|Thương mại th...",0.602151,1.0,WTO là tên sau của tổ chức sau?\n|.Tổ chức Thư...,WTO là một tên viết tắt của tổ chức | Thương m...,0.917165
1,"""Nhằm thúc đẩy sự phát triển kinh tế - xã hội ...","""Trong bối cảnh toàn cầu hóa, các nước đang ph...",0.745407,1.0,Để thúc đẩy phát triển kinh tế xã hội của các ...,"Trong bối cảnh toàn cầu hóa, các nước đang phá...",0.958279
2,"""Đặc điểm nào không đúng với đặc điểm dân cư H...","""Đặc điểm nào sau đây không đúng với đặc điểm ...",0.682540,1.0,Những đặc điểm nào không đúng với đặc điểm dân...,Những đặc điểm nào sau đây không đúng với các ...,0.946972
3,"""Yếu tố tự nhiên nào vừa là khó khăn, nhưng đồ...","""Yếu tố tự nhiên nào vừa là khó khăn nhưng đồn...",0.715556,1.0,"Yếu tố tự nhiên nào cả khó khăn, nhưng cũng th...","Những yếu tố tự nhiên là khó khăn, nhưng cũng ...",0.973532
4,"""Một trong những thuận lợi của EU khi hình thà...","""Một trong những thuận lợi của EU khi hình thà...",0.642659,1.0,Một trong những lợi thế của EU khi hình thành ...,Một trong những lợi ích của EU trong quá trình...,0.929395
...,...,...,...,...,...,...,...
549,"""Nhận xét nào sau đây đúng với họat động ngoại...","""Ý nào sau đây đúng với họat động ngoại thương...",0.944030,1.0,Những ý kiến nào sau đây là đúng với hoạt động...,Phát biểu nào sau đây là đúng đối với thương m...,0.921619
550,"""Hạn chế chủ yếu nhất về điều kiện tự nhiên và...","""Khó khắn lớn nhất về điều kiện tự nhiên và tà...",0.898230,1.0,Giới hạn quan trọng nhất về điều kiện tự nhiên...,Trò đùa lớn nhất về điều kiện tự nhiên của Nhậ...,0.966490
551,"""Mùa đông kéo dài, lạnh và có nhiều tuyết là đ...","""Mùa đông kéo dài, lạnh và có nhiều tuyết là đ...",0.762626,1.0,"Mùa đông kéo dài, lạnh và có rất nhiều tuyết l...","Các phần mùa đông, cái lạnh và có rất nhiều tu...",0.927692
552,"""Sự khác biệt giữa tháp thu hẹp với tháp mở rộ...","""Sự khác biệt giữa tháp mở rộng với tháp thu h...",0.676190,1.0,Sự khác biệt giữa tòa tháp hẹp với tháp mở rộn...,Sự khác biệt giữa tháp mở rộng với tháp hẹp là...,0.898050


In [5]:
df300 = pd.read_csv('Similarity-to-confirm_300.csv', encoding='utf-8')
df800 = pd.read_csv('Similarity-to-confirm-800.csv', encoding='utf-8')
df300.drop('Unnamed: 4',
  axis='columns', inplace=True)
df800.drop('manual test', axis='columns', inplace=True)
df800.drop('Note', axis='columns', inplace=True)
df800 = df800.rename(columns={'Yes/No': 'sem_sim'})
def yes_to_1(yes_no_na):
  if (yes_no_na == 'Y'):
    return 1
  elif (yes_no_na == 'N'):
    return 0
  else:
    return ''
df800.sem_sim = df800['sem_sim'].map(yes_to_1)

In [6]:
df_ = df300.append(df800, sort=False, ignore_index=True)
df_ = df_.drop_duplicates()
df_ = df_[df_.sem_sim.notna()]
df_

,question_1,question_2,similarity,sem_sim
598,"""Mức chi tiêu bình quân của mỗi lượt du khách ...","""Mức chi tiêu bình quân của mỗi lượt du khách ...",0.948148,0
599,"""Mức chi tiêu bình quân của mỗi lượt du khách ...","""Mức chi tiêu bình quân của mỗi lượt du khách ...",0.970588,0
600,"""Cuộc cách mạng khoa học nghệ hiện đại làm xuấ...","""Cuộc cách mạng khoa học nghệ hiện đại làm xuấ...",0.761905,0
601,"""WTO là tên viết tắt của tổ chức nào sau đây?\...","""WTO là tên viết tắt của tổ chức|Thương mại th...",0.602151,1
602,"""Các công ti xuyên quốc gia có vai trò ngày cà...","""Các công ti xuyên quốc gia có đặc điểm nào dư...",0.624473,0
...,...,...,...,...
2425,"""Cho biết nhận xét nào sau đây là đúng|Nhóm nư...","""Hãy cho biết nhận xét nào sau đây là đúng.|Nh...",0.630108,
2426,"""Tỉ suất sinh thô của thế giới năm 2015 là 20%...","""Tỉ suất tử thô của nhóm nước đang phát triển ...",0.626959,
2427,"""Thông thường, nhóm tuổi từ 0 – 14 tuổi được g...","""Thông thường, nhóm tuổi từ 15 – 59 tuổi (hoặc...",0.786982,
2428,"""Thông thường, nhóm tuổi từ 0 – 14 tuổi được g...","""Thông thường, nhóm tuổi từ 60 tuổi (hoặc 65 t...",0.784091,


In [7]:
df_=df_.rename(columns={'question_1': 'sentence1', 'question_2': 'sentence2', 'sem_sim': 'label'})

In [8]:
df_labeled = df_[df_.label==1].append(df_[df_.label==0], sort=False, ignore_index=True)
# df_labeled['idx'] = df_labeled.index
df_labeled.label = df_labeled.apply(lambda x: int(x.label), axis=1)
df_labeled

,sentence1,sentence2,similarity,label
0,"""WTO là tên viết tắt của tổ chức nào sau đây?\...","""WTO là tên viết tắt của tổ chức|Thương mại th...",0.602151,1
1,"""Nhằm thúc đẩy sự phát triển kinh tế - xã hội ...","""Trong bối cảnh toàn cầu hóa, các nước đang ph...",0.745407,1
2,"""Đặc điểm nào không đúng với đặc điểm dân cư H...","""Đặc điểm nào sau đây không đúng với đặc điểm ...",0.682540,1
3,"""Yếu tố tự nhiên nào vừa là khó khăn, nhưng đồ...","""Yếu tố tự nhiên nào vừa là khó khăn nhưng đồn...",0.715556,1
4,"""Một trong những thuận lợi của EU khi hình thà...","""Một trong những thuận lợi của EU khi hình thà...",0.642659,1
...,...,...,...,...
975,"""Nhiệt độ không khí thay đổi theo bờ Đông và b...","""Tại sao nhiệt độ không khí thay đổi theo bờ đ...",0.954023,0
976,"""Vì sao khu vực chịu ảnh hưởng của gió mùa thư...","""Khu vực chịu ảnh hưởng của gió mùa thường có ...",0.959259,0
977,"""Những vùng có tỉ trọng giá trị sản xuất công ...","""Những vùng có tỉ trọng giá trị sản xuất công ...",0.755556,0
978,""" Sản lượng dầu thô khai thác tăng không phải...","""Ở Đông Nam Bộ, Sản lượng dầu thô khai thác tă...",0.902564,0


In [9]:
!pip install sklearn
from sklearn.model_selection import train_test_split

In [10]:
df_train, df_test = train_test_split(df_labeled, test_size=0.2, random_state = 8, shuffle=True)
df_train, df_val = train_test_split(df_labeled, test_size=0.25, random_state = 8)

In [11]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(735, 4)
(245, 4)
(196, 4)


In [12]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

In [13]:
df_train

,sentence1,sentence2,similarity,label
0,"""Cho các thành tựu: . I. Tạo chủng vi khuẩn E....","""Cho các thành tựu: . (1). Tạo chủng vi khuẩn ...",0.864333,0
1,"""Để thể hiện cơ cấu sản lượng thực và cơ cấu s...","""Để thể hiện được tỉ trọng GDP, số dân của EU ...",0.625668,0
2,"""Nhận xét nào dưới đây đúng về đặc điểm phân b...","""Nhận định nào sau đây không đúng với đặc điểm...",0.747440,0
3,"""Nhiệt độ không khí thay đổi theo bờ Đông và b...","""Tại sao nhiệt độ không khí thay đổi theo bờ đ...",0.954023,1
4,"""Ý nào dưới đây là tác động tiêu cực của con n...","""Ý nào dưới đây là tác động tích cực của con n...",0.642534,0
...,...,...,...,...
730,"""Từ năm 1946 đến 1980, đã ba lần Quốc hội thôn...","""Từ năm 1946 đến 1980, đã ba lần Quốc hội thôn...",0.989399,1
731,"""""Lần đầu tiên trong lịch sử, một nước thuộc đ...","""“Lần đầu tiên trong lịch sử, một nước thuộc đ...",0.779904,1
732,"""Từ Ô-đét-xa đến Mum-bai, đi vòng qua châu phi...","""Từ Niu-Iooc đến Xít-ni, đi vòng qua Nam Mĩ là...",0.744048,0
733,"""Hình thức tổ chức lãnh thổ công nghiệp đơn gi...","""Hình thức tổ chức lãnh thổ công nghiệp ở trìn...",0.650307,0


# Classes and functions

In [ ]:
!pip3 install fairseq
!pip3 install fastbpe
!pip3 install vncorenlp

In [15]:
!wget https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
!tar -xzvf PhoBERT_base_fairseq.tar.gz
!ls PhoBERT_base_fairseq

--2021-12-23 05:38:33--  https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
Resolving public.vinai.io (public.vinai.io)... 65.8.49.115, 65.8.49.107, 65.8.49.47, ...
Connecting to public.vinai.io (public.vinai.io)|65.8.49.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1243308020 (1.2G) [application/x-tar]
Saving to: ‘PhoBERT_base_fairseq.tar.gz.2’

PhoBERT_base_fairse 100%[===================>]   1.16G  7.50MB/s    in 2m 59s  

2021-12-23 05:41:33 (6.63 MB/s) - ‘PhoBERT_base_fairseq.tar.gz.2’ saved [1243308020/1243308020]

PhoBERT_base_fairseq/
PhoBERT_base_fairseq/bpe.codes
PhoBERT_base_fairseq/model.pt
PhoBERT_base_fairseq/dict.txt
bpe.codes  dict.txt  model.pt


In [16]:
from fairseq.models.roberta import RobertaModel
phoBERT = RobertaModel.from_pretrained('PhoBERT_base_fairseq', checkpoint_file='model.pt')

In [17]:
from fairseq.data.encoders.fastbpe import fastBPE

# Khởi tạo Byte Pair Encoding cho PhoBERT
class BPE():
  bpe_codes = 'PhoBERT_base_fairseq/bpe.codes'

args = BPE()
phoBERT.bpe = fastBPE(args) #Incorporate the BPE encoder into PhoBERT
tokens = phoBERT.encode('Tôn Ngộ Không phò Đường Tăng đi Tây Trúc thỉnh kinh')
print('tokens list : ', tokens)
# Decode ngược lại thành câu từ chuỗi index token
phoBERT.decode(tokens)  # 'Hello world!'

tokens list :  tensor([    0, 11623, 31433,   453, 44334,  2080,  5922,    57,   934,  8181,
        31686,  3078,     2])


'Tôn Ngộ Không phò Đường Tăng đi Tây Trúc thỉnh kinh'

In [14]:
from transformers import BertTokenizer, BertModel

In [15]:
class CustomDataset(Dataset):

    def __init__(self, data, maxlen, with_labels=True, bert_model='bert-base-uncased'):

        self.data = data  # pandas dataframe
        #Initialize the tokenizer
        self.tokenizer = BertTokenizer.from_pretrained(bert_model)  

        self.maxlen = maxlen
        self.with_labels = with_labels 

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):

        # Selecting sentence1 and sentence2 at the specified index in the data frame
        sent1 = str(self.data.loc[index, 'sentence1'])
        sent2 = str(self.data.loc[index, 'sentence2'])

        # Tokenize the pair of sentences to get token ids, attention masks and token type ids
        encoded_pair = self.tokenizer(sent1, sent2, 
                                      padding='max_length',  # Pad to max_length
                                      truncation=True,  # Truncate to max_length
                                      max_length=self.maxlen,  
                                      return_tensors='pt')  # Return torch.Tensor objects
        
        token_ids = encoded_pair['input_ids'].squeeze(0)  # tensor of token ids
        attn_masks = encoded_pair['attention_mask'].squeeze(0)  # binary tensor with "0" for padded values and "1" for the other values
        token_type_ids = encoded_pair['token_type_ids'].squeeze(0)  # binary tensor with "0" for the 1st sentence tokens & "1" for the 2nd sentence tokens

        if self.with_labels:  # True if the dataset has labels
            label = self.data.loc[index, 'label']
            return token_ids, attn_masks, token_type_ids, label  
        else:
            return token_ids, attn_masks, token_type_ids

In [19]:
# Load the model in fairseq
from fairseq.models.roberta import RobertaModel
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary

phoBERT_cls = RobertaModel.from_pretrained('PhoBERT_base_fairseq', checkpoint_file='model.pt')
phoBERT_cls.eval()  # disable dropout (or leave in train mode to finetune

# Load BPE
class BPE():
  bpe_codes = 'PhoBERT_base_fairseq/bpe.codes'

args = BPE()
phoBERT_cls.bpe = fastBPE(args) #Incorporate the BPE encoder into PhoBERT

# Add header cho classification với số lượng classes = 2
phoBERT_cls.register_classification_head('new_task', num_classes=2)
# tokens = 'Học_sinh được nghỉ học bắt đầu từ tháng 3 do ảnh hưởng của dịch covid-19'
# token_idxs = phoBERT_cls.encode(tokens)
# logprobs = phoBERT_cls.predict('new_task', token_idxs)  # tensor([[-1.1050, -1.0672, -1.1245]], grad_fn=<LogSoftmaxBackward>)
# logprobs

In [16]:
class SentencePairClassifier(nn.Module):

    def __init__(self, bert_model="bert-base-uncased", freeze_bert=False):
        super(SentencePairClassifier, self).__init__()
        #  Instantiating BERT-based model object
        self.bert_layer = BertModel.from_pretrained(bert_model, return_dict=False)
        # self.bert_layer.bpe = fastBPE(args)

        #  Fix the hidden-state size of the encoder outputs (If you want to add other pre-trained models here, search for the encoder output size)
        # if bert_model == "albert-base-v2":  # 12M parameters
        #     hidden_size = 768
        # elif bert_model == "albert-large-v2":  # 18M parameters
        #     hidden_size = 1024
        # elif bert_model == "albert-xlarge-v2":  # 60M parameters
        #     hidden_size = 2048
        # elif bert_model == "albert-xxlarge-v2":  # 235M parameters
        #     hidden_size = 4096
        # elif bert_model == "bert-base-uncased": # 110M parameters
        #     hidden_size = 768

        # hidden_size = 135
        hidden_size = 768

        # Freeze bert layers and only train the classification layer weights
        if freeze_bert:
            for p in self.bert_layer.parameters():
                p.requires_grad = False

        # Classification layer
        self.cls_layer = nn.Linear(hidden_size, 1)

        self.dropout = nn.Dropout(p=0.1)

    @autocast()  # run in mixed precision
    def forward(self, input_ids, attn_masks, token_type_ids):
        '''
        Inputs:
            -input_ids : Tensor  containing token ids
            -attn_masks : Tensor containing attention masks to be used to focus on non-padded values
            -token_type_ids : Tensor containing token type ids to be used to identify sentence1 and sentence2
        '''

        # Feeding the inputs to the BERT-based model to obtain contextualized representations
        # cont_reps, pooler_output = self.bert_layer(input_ids, attn_masks, token_type_ids)
        cont_reps, pooler_output = self.bert_layer(input_ids, attn_masks, token_type_ids)

        # Feeding to the classifier layer the last layer hidden-state of the [CLS] token further processed by a
        # Linear Layer and a Tanh activation. The Linear layer weights were trained from the sentence order prediction (ALBERT) or next sentence prediction (BERT)
        # objective during pre-training.
        logits = self.cls_layer(self.dropout(pooler_output))
        
        return logits

In [17]:
def set_seed(seed):
    """ Set all seeds to make results reproducible """
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    

def evaluate_loss(net, device, criterion, dataloader):
    net.eval()

    mean_loss = 0
    count = 0

    with torch.no_grad():
        for it, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm(dataloader)):
            seq, attn_masks, token_type_ids, labels = \
                seq.to(device), attn_masks.to(device), token_type_ids.to(device), labels.to(device)
            logits = net(seq, attn_masks, token_type_ids)
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            count += 1

    return mean_loss / count

In [18]:
print("Creation of the models' folder...")
!mkdir models

Creation of the models' folder...
mkdir: cannot create directory ‘models’: File exists


In [19]:
def train_bert(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate):

    best_loss = np.Inf
    best_ep = 1
    nb_iterations = len(train_loader)
    print_every = nb_iterations // 5  # print the training loss 5 times per epoch
    iters = []
    train_losses = []
    val_losses = []

    scaler = GradScaler()

    for ep in range(epochs):

        net.train()
        running_loss = 0.0
        for it, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm(train_loader)):

            # Converting to cuda tensors
            seq, attn_masks, token_type_ids, labels = \
                seq.to(device), attn_masks.to(device), token_type_ids.to(device), labels.to(device)
    
            # Enables autocasting for the forward pass (model + loss)
            with autocast():
                # Obtaining the logits from the model
                logits = net(seq, attn_masks, token_type_ids)

                # Computing loss
                loss = criterion(logits.squeeze(-1), labels.float())
                loss = loss / iters_to_accumulate  # Normalize the loss because it is averaged

            # Backpropagating the gradients
            # Scales loss.  Calls backward() on scaled loss to create scaled gradients.
            scaler.scale(loss).backward()

            if (it + 1) % iters_to_accumulate == 0:
                # Optimization step
                # scaler.step() first unscales the gradients of the optimizer's assigned params.
                # If these gradients do not contain infs or NaNs, opti.step() is then called,
                # otherwise, opti.step() is skipped.
                scaler.step(opti)
                # Updates the scale for next iteration.
                scaler.update()
                # Adjust the learning rate based on the number of iterations.
                lr_scheduler.step()
                # Clear gradients
                opti.zero_grad()


            running_loss += loss.item()

            if (it + 1) % print_every == 0:  # Print training loss information
                print()
                print("Iteration {}/{} of epoch {} complete. Loss : {} "
                      .format(it+1, nb_iterations, ep+1, running_loss / print_every))

                running_loss = 0.0


        val_loss = evaluate_loss(net, device, criterion, val_loader)  # Compute validation loss
        print()
        print("Epoch {} complete! Validation Loss : {}".format(ep+1, val_loss))

        if val_loss < best_loss:
            print("Best validation loss improved from {} to {}".format(best_loss, val_loss))
            print()
            net_copy = copy.deepcopy(net)  # save a copy of the model
            best_loss = val_loss
            best_ep = ep + 1

    # Saving the model
    path_to_model='models/{}_lr_{}_val_loss_{}_ep_{}.pt'.format(bert_model, lr, round(best_loss, 5), best_ep)
    torch.save(net_copy.state_dict(), path_to_model)
    print("The model has been saved in {}".format(path_to_model))

    del loss
    torch.cuda.empty_cache()

# Parameter

In [20]:
bert_model = "bert-base-uncased"  # 'albert-base-v2', 'albert-large-v2', 'albert-xlarge-v2', 'albert-xxlarge-v2', 'bert-base-uncased', ...
freeze_bert = False  # if True, freeze the encoder weights and only update the classification layer weights
maxlen = 256  # maximum length of the tokenized input sentence pair : if greater than "maxlen", the input is truncated and else if smaller, the input is padded
bs = 8  # batch size
iters_to_accumulate = 2  # the gradient accumulation adds gradients over an effective batch of size : bs * iters_to_accumulate. If set to "1", you get the usual batch size
lr = 2e-5  # learning rate
epochs = 4  # number of training epochs

# Training and Validation

In [21]:
#  Set all seeds to make reproducible results
set_seed(1)

# Creating instances of training and validation set
print("Reading training data...")
train_set = CustomDataset(df_train, maxlen, bert_model)
print("Reading validation data...")
val_set = CustomDataset(df_val, maxlen, bert_model)
# Creating instances of training and validation dataloaders
train_loader = DataLoader(train_set, batch_size=bs, num_workers=5,shuffle=True)
val_loader = DataLoader(val_set, batch_size=bs, num_workers=5,shuffle=True)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = SentencePairClassifier(bert_model, freeze_bert=freeze_bert)

if torch.cuda.device_count() > 1:  # if multiple GPUs
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)

net.to(device)

criterion = nn.BCEWithLogitsLoss()
opti = AdamW(net.parameters(), lr=lr, weight_decay=1e-2)
num_warmup_steps = 0 # The number of steps for the warmup phase.
num_training_steps = epochs * len(train_loader)  # The total number of training steps
t_total = (len(train_loader) // iters_to_accumulate) * epochs  # Necessary to take into account Gradient accumulation
lr_scheduler = get_linear_schedule_with_warmup(optimizer=opti, num_warmup_steps=num_warmup_steps, num_training_steps=t_total)

train_bert(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate)

Reading training data...
Reading validation data...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
 20%|█▉        | 18/92 [00:26<01:45,  1.43s/it]


Iteration 18/92 of epoch 1 complete. Loss : 0.31382705602380967 


 39%|███▉      | 36/92 [00:51<01:18,  1.40s/it]


Iteration 36/92 of epoch 1 complete. Loss : 0.30450642026133007 


...


Iteration 72/92 of epoch 4 complete. Loss : 0.1606680622531308 


 98%|█████████▊| 90/92 [02:05<00:02,  1.40s/it]


Iteration 90/92 of epoch 4 complete. Loss : 0.1508995189021031 


100%|██████████| 31/31 [00:15<00:00,  2.05it/s]



Epoch 4 complete! Validation Loss : 0.34149481860860703
Best validation loss improved from 0.36755204393017676 to 0.34149481860860703

The model has been saved in models/bert-base-uncased_lr_2e-05_val_loss_0.34149_ep_4.pt


# Prediction

In [ ]:
print("Creation of the models' folder...")
!mkdir models

Creation of the models' folder...
mkdir: cannot create directory ‘models’: File exists


In [22]:
def get_probs_from_logits(logits):
    """
    Converts a tensor of logits into an array of probabilities by applying the sigmoid function
    """
    probs = torch.sigmoid(logits.unsqueeze(-1))
    return probs.detach().cpu().numpy()

def test_prediction(net, device, dataloader, with_labels=True, result_file="results/output.txt"):
    """
    Predict the probabilities on a dataset with or without labels and print the result in a file
    """
    net.eval()
    w = open(result_file, 'w')
    probs_all = []

    with torch.no_grad():
        if with_labels:
            for seq, attn_masks, token_type_ids, _ in tqdm(dataloader):
                seq, attn_masks, token_type_ids = seq.to(device), attn_masks.to(device), token_type_ids.to(device)
                logits = net(seq, attn_masks, token_type_ids)
                probs = get_probs_from_logits(logits.squeeze(-1)).squeeze(-1)
                probs_all += probs.tolist()
        else:
            for seq, attn_masks, token_type_ids in tqdm(dataloader):
                seq, attn_masks, token_type_ids = seq.to(device), attn_masks.to(device), token_type_ids.to(device)
                logits = net(seq, attn_masks, token_type_ids)
                probs = get_probs_from_logits(logits.squeeze(-1)).squeeze(-1)
                probs_all += probs.tolist()

    w.writelines(str(prob)+'\n' for prob in probs_all)
    w.close()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [24]:
path_to_model = '/content/models/bert-base-uncased_lr_2e-05_val_loss_0.34149_ep_4.pt'  

path_to_output_file = 'results/output.txt'

print("Reading test data...")
test_set = CustomDataset(df_test, maxlen, bert_model)
test_loader = DataLoader(test_set, batch_size=bs, num_workers=5)

model = SentencePairClassifier(bert_model)
if torch.cuda.device_count() > 1:  # if multiple GPUs
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

print()
print("Loading the weights of the model...")
model.load_state_dict(torch.load(path_to_model))
model.to(device)

print("Predicting on test data...")
test_prediction(net=model, device=device, dataloader=test_loader, with_labels=True,  # set the with_labels parameter to False if your want to get predictions on a dataset without labels
                result_file=path_to_output_file)
print()
print("Predictions are available in : {}".format(path_to_output_file))

Reading test data...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Loading the weights of the model...
Predicting on test data...


100%|██████████| 25/25 [00:12<00:00,  1.98it/s]


Predictions are available in : results/output.txt


# Evaluation

In [25]:
path_to_output_file = 'results/output.txt'  # path to the file with prediction probabilities

labels_test = df_test['label']  # true labels

probs_test = pd.read_csv(path_to_output_file, header=None)[0]  # prediction probabilities
threshold = 0.5   # you can adjust this threshold for your own dataset
preds_test=(probs_test>=threshold).astype('uint8') # predicted labels using the above fixed threshold

metric = load_metric("glue", "mrpc")

https://raw.githubusercontent.com/huggingface/datasets/1.0.1/metrics/glue/glue.py not found in cache or force_download set to True, downloading to /root/.cache/huggingface/datasets/tmppdro2kgm


Downloading:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

storing https://raw.githubusercontent.com/huggingface/datasets/1.0.1/metrics/glue/glue.py in cache at /root/.cache/huggingface/datasets/50d5843bbbbd80c47809bc76a5b03c0fd87d068509b0060103ae8182e4f5cfb9.ec871b06a00118091ec63eff0a641fddcb8d3c7cd52e855bbb2be28944df4b82.py
creating metadata file for /root/.cache/huggingface/datasets/50d5843bbbbd80c47809bc76a5b03c0fd87d068509b0060103ae8182e4f5cfb9.ec871b06a00118091ec63eff0a641fddcb8d3c7cd52e855bbb2be28944df4b82.py
Checking /root/.cache/huggingface/datasets/50d5843bbbbd80c47809bc76a5b03c0fd87d068509b0060103ae8182e4f5cfb9.ec871b06a00118091ec63eff0a641fddcb8d3c7cd52e855bbb2be28944df4b82.py for additional imports.
Creating main folder for metric https://raw.githubusercontent.com/huggingface/datasets/1.0.1/metrics/glue/glue.py at /root/.cache/huggingface/modules/datasets_modules/metrics/glue
Creating specific version folder for metric https://raw.githubusercontent.com/huggingface/datasets/1.0.1/metrics/glue/glue.py at /root/.cache/huggingface/mod

In [26]:
preds_test

0      0
1      1
2      0
3      0
4      0
      ..
191    0
192    0
193    0
194    0
195    0
Name: 0, Length: 196, dtype: uint8

In [27]:
# Compute the accuracy and F1 scores
metric._compute(predictions=preds_test, references=labels_test)

{'accuracy': 0.8571428571428571, 'f1': 0.7454545454545455}

In [28]:
from sklearn.metrics import accuracy_score
accuracy_train = accuracy_score(preds_test, labels_test)

In [29]:
accuracy_train

0.8571428571428571